# Incident Neutron Energy Spectrum After First Wall Effects

In [1]:
import openmc
import os, sys
from pathlib import Path
import numpy as np

# Import helper functions
sys.path.insert(0, f"{Path.cwd()}/helper")
from utilities import *

In [2]:
""" MATERIALS """
""" TUNGSTEN from ball"""
tungsten = openmc.Material(name='W')
tungsten.add_element('O',5/1e6,percent_type='wo')
tungsten.add_element('N',5/1e6,percent_type='wo')
tungsten.add_element('C',5/1e6,percent_type='wo')
tungsten.add_element('Na',4/1e6,percent_type='wo')
tungsten.add_element('K',2.5/1e6,percent_type='wo')
tungsten.add_element('Al',3/1e6,percent_type='wo')
tungsten.add_element('Ca',0.5/1e6,percent_type='wo')
tungsten.add_element('Cr',0.5/1e6,percent_type='wo')
tungsten.add_element('Cu',0.5/1e6,percent_type='wo')
tungsten.add_element('Fe',5/1e6,percent_type='wo')

tungsten.add_element('W',1-(5+5+5+4+2.5+3+0.5+0.5+0.5+5)/1e6,percent_type='wo')
tungsten.set_density('g/cm3',19.3)
#tungsten.volume = 65636 #Wedge vol
tungsten.depletable = False

""" V-4Cr-4Ti """
vcrti = openmc.Material(name='V-4Cr-4Ti VV')
vcrti.depletable = False

vcrti.add_element('Cr',0.04,percent_type='wo')
vcrti.add_element('Ti',0.04,percent_type='wo')

vcrti.add_element('C',56/1e6,percent_type='wo')
vcrti.add_element('O',181/1e6,percent_type='wo')
vcrti.add_element('N',103/1e6,percent_type='wo')
vcrti.add_element('B',7/1e6,percent_type='wo')
vcrti.add_element('Na',17/1e6,percent_type='wo')
vcrti.add_element('Mg',0.5/1e6,percent_type='wo')
vcrti.add_element('Al',119/1e6,percent_type='wo')
vcrti.add_element('Si',280/1e6,percent_type='wo')
vcrti.add_element('Mn',0.5/1e6,percent_type='wo')
vcrti.add_element('Fe',80/1e6,percent_type='wo')
vcrti.add_element('Ni',13/1e6,percent_type='wo')
vcrti.add_element('Cu',4/1e6,percent_type='wo')
vcrti.add_element('V',1-0.04-0.04-(56+181+103+7+17+0.5+119+280+0.5+80+13+4)/1e6,percent_type='wo')
vcrti.set_density('g/cm3',6.05) #This density value is sus and needs a good source

materials = openmc.Materials([tungsten,vcrti])

In [3]:
"""GEOMETRY"""
'''first wall slab thickness approximations from balls simplified model
but we are ignoring the effects of imbeded structural components within the flibe mixture
such as the channel structures, RF heating, and vacuum systems.'''

tung_thickness = 0.3 #cm
vcrti_thickness = 1

r1 = openmc.Sphere(r=tung_thickness)  # inner radius 10 cm
r2 = openmc.Sphere(r=vcrti_thickness+tung_thickness, boundary_type='vacuum')

tung_region = -r1
vcrti_region = +r1 & -r2

tung_cell = openmc.Cell(fill=tungsten, region=tung_region)
vcrti_cell = openmc.Cell(fill=vcrti, region=vcrti_region)

geometry = openmc.Geometry([tung_cell, vcrti_cell])

In [4]:
""" CROSS SECTIONS 
Temporary solution for finding xs files between WSL and Ubuntu on Computing Cluster without editing PATH --ppark 2025-06-28 
"""
xs_path_ubuntu = '/opt/openmc_data/endfb-viii.0-hdf5/cross_sections.xml'
xs_path_wsl   = '/mnt/c/openmc/data/endfb-viii.0-hdf5/cross_sections.xml'
if os.path.isfile(xs_path_ubuntu):
    materials.cross_sections = xs_path_ubuntu # use this on Zotacs --ppark
elif os.path.isfile(xs_path_wsl):
    materials.cross_sections = xs_path_wsl
else:
    print(f"Error finding cross section XML!")
    # sys.exit(2)

In [5]:
""" TALLIES """
tallies = openmc.Tallies() # initialize

# Filters
E_bin_edges = logspace_per_decade(1e-5, 20e6, 100) # './helpers/utilities.py'
energy_filter = openmc.EnergyFilter(E_bin_edges)

surface_filter = openmc.SurfaceFilter(r2)

# Current tally to measure outgoing neutron spectrum
out_tally = openmc.Tally(name='outgoing_spectrum')
out_tally.filters = [surface_filter, energy_filter]
out_tally.scores = ['current']

tallies = openmc.Tallies([out_tally])
tallies.export_to_xml()



In [6]:
""" SETTINGS """
settings = openmc.Settings()

""" Source
Isotropic 14.07 MeV plane source at left side surface
"""
""" Run type """
settings = openmc.Settings()
settings.run_mode = 'fixed source'
settings.particles = int(1e6)
settings.batches = 100

source = openmc.IndependentSource()
source.space = openmc.stats.Point((0,0,0)) 
source.angle = openmc.stats.Isotropic()     
source.energy = openmc.stats.Discrete([14.07e6], [1.0])  

settings.source = source


In [7]:
model = openmc.model.Model(geometry, materials, settings, tallies)
model.export_to_model_xml("./model_FirstWall_7_14/")
results_NSlow = model.run(cwd="./model_FirstWallTest_7_14/")  

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################